# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
from copy import copy
from functools import partial
from keras.models import *
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.constraints import max_norm
from keras.preprocessing.text import *
from keras.optimizers import *
from keras.metrics import *
from keras.callbacks import *
from keras import Input
from keras.utils import to_categorical
import numpy as np
import pandas as pd
import time
from collections import namedtuple
import tensorflow as tf

%matplotlib inline
np.random.seed(7)

Using TensorFlow backend.


First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 101 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:1001]

"Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverything was in confusion in the Oblonskys' house. The wife had\ndiscovered that the husband was carrying on an intrigue with a French\ngirl, who had been a governess in their family, and she had announced to\nher husband that she could not go on living in the same house with him.\nThis position of affairs had now lasted three days, and not only the\nhusband and wife themselves, but all the members of their family and\nhousehold, were painfully conscious of it. Every person in the house\nfelt that there was no sense in their living together, and that the\nstray people brought together by chance in any inn had more in common\nwith one another than they, the members of the family and household of\nthe Oblonskys. The wife did not leave her own room, the husband had not\nbeen at home for three days. The children ran wild all over the house;\nthe English governess quarreled with the housekeep

And we can see the characters encoded as integers.

In [4]:
encoded[:101]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70, 63])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [6]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches*characters_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape(batch_size,-1)
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y_uptolast = arr[:, n+1:n+n_steps+1]
        # last y will be one element short, so need to preset
        # the dimensions by filling in with zeros
        # for missing last elements
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_uptolast.shape[1]] = y_uptolast
        
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 100, 50)
x, y = next(batches)

In [8]:
print('x\n', x)
print('\ny\n', y)

x
 [[31 64 57 ...  1 77 70]
 [59 71 68 ... 65 76 76]
 [ 1 66 77 ... 57 65 60]
 ...
 [71  1 76 ... 70  1 64]
 [61  1 79 ... 78 65 76]
 [ 1 65 70 ...  1 69 61]]

y
 [[64 57 72 ... 77 70 64]
 [71 68 57 ... 76 76 68]
 [66 77 75 ... 65 60  1]
 ...
 [ 1 76 64 ...  1 64 61]
 [ 1 79 57 ... 65 76 57]
 [65 70 76 ... 69 61 11]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

### Example

In [ ]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

print(x.shape, y.shape)
model.summary()

# model.fit(x, y,batch_size=128,epochs=60,callbacks=[print_callback])

### Keras Implementation

In [9]:
#training x and y
df = pd.DataFrame(encoded)
df = pd.concat([df, df.shift(1)], axis=1)
df.dropna(inplace=True)
values = df.values
train_x, train_y = values[:, 0], values[:, 1]
train_x= to_categorical(train_x[:1982464]).reshape(-1,83).astype('bool')
train_y= to_categorical(train_y[:1982464]).reshape(-1,83).astype('bool')

train_x.astype('int8').shape, train_y.astype('int8').shape

#encoded array
oht = to_categorical(encoded).astype('int8')

#generator
def genoht(oht_arr, tsteps):
    
    enc_vsize = oht_arr.shape[-1]
    chars_batch = 1*tsteps
    n_batches  = len(oht_arr)//chars_batch
    
    #clip oht_arr
    oht_arr = oht_arr[:n_batches*chars_batch]
    
    #reshape with features
    oht_arr = oht_arr.reshape(1,-1,enc_vsize)
    
    for n in range(0, oht_arr.shape[1], tsteps):
        x = oht_arr[:, n:n+tsteps]
        ytemp = oht_arr[:, n+1:n+tsteps+1]
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:ytemp.shape[1]] = ytemp
        # one hot x 
        x =  x.astype('int8').reshape(-1,1,83)
        # sparse y
        y = np.argmax(y.astype('int8').reshape(-1,83), axis=1)
        y = y.astype('int8').reshape(-1,1)
        
        yield x,y
        
oht_genfn  = genoht(oht,256)

In [10]:
x, y  = next(oht_genfn)
x.shape, y.shape

((256, 1, 83), (256, 1))

In [61]:
class CharRNN():
    
    def __init__(self, num_classes, batch_size=1024, num_steps=128, 
                 lstm_size=512, learning_rate=0.001,
                 sampling=False, dropout=0.5, stateful=True):
        
        #config
        self.evl = 32
        self.lstm_units = int(lstm_size)
        self.lr = learning_rate
        self.sampling = sampling
        self.classes = num_classes
        self.dropval = dropout
        self.bsize = batch_size
        self.stepsnum = num_steps
        self.onehot_arr = self.onehot()        # categorically encoded array
        self.tr_x = train_x
        self.tr_y = train_y
        self.chkptname = "checkpoints/keras_lstmm_weights-best.hdf5"
        #alt-fns
        self.timestampfn = lambda :str(list(['iter:',self.count,
                                'lstm:',self.lstm_units,self.layers,
                                '@:',int(time.time()//2)]))
        self.expand = lambda x : np.expand_dims(x, axis=0)
        #model definitions        
        #shape
        self.batchSize = (self.stepsnum,1,83)
        self.inp_Size = (1,83)
        # state
        self.count = 0
        self.stateful = stateful
        #modules
        self.dropout = partial(Dropout, rate=self.dropval)
        self.Embed = partial(Embedding, )
        self.cnn1D = partial(Conv1D, kernel_size=8, strides=1, activation='relu')
        self.cnnpool = partial(AveragePooling1D, pool_size=4)
        self.GRU = partial(CuDNNGRU, units=self.lstm_units, stateful=self.stateful)
        self.Dense = partial(Dense, activation='softmax')
        self.timedist_wrap = partial(TimeDistributed, )
        self.flatten = partial(Flatten, )
        # model instance
        self.model = self.build_model()
        #callbacks
        self.metric = mse
        self.checkpoint = ModelCheckpoint(self.chkptname, monitor='loss',
                                          verbose=1, save_best_only=True)
        
        self.LROP = ReduceLROnPlateau(monitor='loss', factor=0.1,
                                      patience=2, verbose=1,mode='min',
                                      min_delta=1e-6,cooldown=0, min_lr=1e-9)
        
        self.EarlyStopping =EarlyStopping(monitor='loss', min_delta=1e-6, 
                                          patience=5, verbose=1, mode='min')
        
        self.callbacks = [self.checkpoint, self.LROP, self.EarlyStopping]
        self.optimizer = SGD(lr=self.lr, momentum=0.9, nesterov=True)
        
    def onehot(self):
        data  = copy.copy(encoded)
        return to_categorical(data).astype('int8')
    
    def hotgen(self):
        self.count += 1
        oht_arr = self.onehot_arr
        enc_vsize = oht_arr.shape[-1]
        chars_batch = 1*self.stepsnum
        n_batches  = len(oht_arr)//chars_batch

        #clip oht_arr
        oht_arr = oht_arr[:n_batches*chars_batch]

        #reshape with features
        oht_arr = oht_arr.reshape(1,-1,enc_vsize)

        for n in range(0, oht_arr.shape[1], self.stepsnum):
            x = oht_arr[:, n:n+self.stepsnum]
            ytemp = oht_arr[:, n+1:n+self.stepsnum+1]
            y = np.zeros(x.shape, dtype=x.dtype)
            y[:,:ytemp.shape[1]] = ytemp
            # one hot x 
            x =  x.astype('int8').reshape(-1,1,83)
            # sparse y
            y = y.astype('int8').reshape(-1,83)
        
            yield x,y
            
    def timeseriesgen(self):
        # experimental - leads to guaranteed oom on tflow
        from keras.preprocessing.sequence import TimeseriesGenerator
        # returns a keras sequence object.
        datagen = TimeseriesGenerator(self.tr_x, self.tr_y, 
                                      length=self.stepsnum, 
                                      batch_size = 1, 
                                      sampling_rate=1,
                                      shuffle=False)
        return datagen
            
            
    def listcallbacks(self):
        return self.callbacks
    
    def build_model(self):
        if self.stateful:
            input_ = Input(batch_shape = self.batchSize)
        else:
            input_ = Input(shape = self.inp_Size)
        flatemb = self.flatten(name='flat_inp')(input_)
        embed = self.Embed(83, 64, input_length=83, name='emb_pre_cnn')(flatemb)
        cnn1D = self.cnn1D(self.lstm_units//8, name='cnn1D_size_8')(embed)
        cnnpool = self.cnnpool(name='cnnpool_size_8')(cnn1D)
        gru_0 = self.GRU(return_sequences=True, name='gru0_post_cnn1D')(cnnpool)
        dropoutL1 = self.dropout(name='drop_gru0')(gru_0)
        gru_1 = self.GRU(return_sequences=True, name='gru_1')(dropoutL1)
        dropoutL2 = self.dropout(name='drop_gru1')(gru_1)
        gru_2 = self.GRU(name='gru2_final')(dropoutL2)
        dropoutL3 = self.dropout(name='drop_gru2_final')(gru_2)
        #flat_predense = self.flatten(name='flat_predense')(dropoutL3)
        dense = self.Dense(self.classes)(dropoutL3)
        lstmm = Model(inputs=input_, outputs=dense)
        return lstmm

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [62]:
train_x.shape, train_y.shape

((1982464, 83), (1982464, 83))

#### Model Configs

In [91]:
def clear_start(clear = False):
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
    if clear:
        K.clear_session()
    
clear_start(True)

#num_layers = 3         # FIXED: Number of LSTM layers
batch_size = 256        # Sequences per batch
num_steps = 128          # Number of sequence steps per batch
lstm_size = 1024         # Size of hidden layers in LSTMs
learning_rate = 1e-6       # Learning rate
keep_prob = 0.8          # Dropout keep probability
epochs = 100

In [92]:
LSTM_RNN = CharRNN(len(np.unique(encoded)), 
                   lstm_size=lstm_size,
                   batch_size = batch_size,
                   num_steps=num_steps, 
                   learning_rate=learning_rate,
                   dropout=keep_prob)

lstm_model = LSTM_RNN.model

lstm_optimizer = SGD(lr = LSTM_RNN.lr, momentum = 0.9, decay = 1e-6, nesterov=True)

lstm_model.compile(loss='categorical_crossentropy',
                   optimizer=lstm_optimizer,
                   metrics=[LSTM_RNN.metric])

print(lstm_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (128, 1, 83)              0         
_________________________________________________________________
flat_inp (Flatten)           (128, 83)                 0         
_________________________________________________________________
emb_pre_cnn (Embedding)      (128, 83, 64)             5312      
_________________________________________________________________
cnn1D_size_8 (Conv1D)        (128, 76, 128)            65664     
_________________________________________________________________
cnnpool_size_8 (AveragePooli (128, 19, 128)            0         
_________________________________________________________________
gru0_post_cnn1D (CuDNNGRU)   (128, 19, 1024)           3545088   
_________________________________________________________________
drop_gru0 (Dropout)          (128, 19, 1024)           0         
__________

In [93]:
x = LSTM_RNN.hotgen()
x,y = next(x)
x.shape, y.shape

((128, 1, 83), (128, 83))

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [94]:
callbacks = LSTM_RNN.callbacks
LSTM_RNN_datagen  = LSTM_RNN.hotgen()

lstm_model.load_weights('checkpoints/keras_lstmm_weights-best.hdf5')

lstm_model.fit_generator(LSTM_RNN_datagen, steps_per_epoch = LSTM_RNN.bsize, 
                         epochs=epochs, shuffle = False,
                         workers= 1, use_multiprocessing=False,
                         callbacks=callbacks)

clear_start(True)

Epoch 1/100
256/256 [==============================] - 20s 78ms/step - loss: 2.5604 - mean_squared_error: 0.0104

Epoch 00001: loss improved from inf to 2.56037, saving model to checkpoints/keras_lstmm_weights-best.hdf5
Epoch 2/100
256/256 [==============================] - 19s 75ms/step - loss: 2.5518 - mean_squared_error: 0.0105

Epoch 00002: loss improved from 2.56037 to 2.55182, saving model to checkpoints/keras_lstmm_weights-best.hdf5
Epoch 3/100
256/256 [==============================] - 20s 76ms/step - loss: 2.5699 - mean_squared_error: 0.0104

Epoch 00003: loss did not improve from 2.55182
Epoch 4/100
256/256 [==============================] - 20s 76ms/step - loss: 2.5208 - mean_squared_error: 0.0104

Epoch 00004: loss improved from 2.55182 to 2.52078, saving model to checkpoints/keras_lstmm_weights-best.hdf5
Epoch 5/100
256/256 [==============================] - 19s 76ms/step - loss: 2.5685 - mean_squared_error: 0.0104

Epoch 00005: loss did not improve from 2.52078
Epoch 6/10

### generation

In [95]:
clear_start(True)

LSTM_RNN = CharRNN(len(np.unique(encoded)), 
                   lstm_size=lstm_size,
                   batch_size = batch_size,
                   num_steps=num_steps, 
                   learning_rate=learning_rate,
                   dropout=keep_prob)

lstm_model = LSTM_RNN.model
lstm_model.compile(loss='categorical_crossentropy',
                   optimizer=lstm_optimizer,
                   metrics=[LSTM_RNN.metric])
lstm_model.load_weights('checkpoints/keras_lstmm_weights-best.hdf5')
weights = lstm_model.get_weights()

LSTMM_pred = CharRNN(len(np.unique(encoded)), 
                   lstm_size=lstm_size,
                   batch_size = 1,
                   num_steps=3, 
                   learning_rate=learning_rate,
                   dropout=0,
                   stateful=True)

lstm_pred_model = LSTMM_pred.model
lstm_pred_model.set_weights(weights=weights)

lstm_pred_model.compile(loss='categorical_crossentropy',
                   optimizer=lstm_optimizer,
                   metrics=[LSTM_RNN.metric]) 

In [96]:
getchar = lambda x, axis: str(vocab[np.asscalar(np.argmax(x,axis=axis).flatten())])

seed  = np.random.randint(0,train_x.shape[0])
seed = train_x[seed].astype('int8').reshape(1,1,83)

res = lstm_pred_model.predict(seed,verbose=1)
print(getchar(seed, 2),'->',getchar(res,1))

ValueError: Cannot feed value of shape (1, 1, 83) for Tensor 'input_2:0', which has shape '(3, 1, 83)'

In [ ]:
# things to define for ze zenerativ loupe
# lstmm.predict(seed.to_onehot(depth=len(vocab))) -> lstm_out
# lstm_out[np.argmax, softmax] -> lstm_char_index
# lstm_char_index -> vocab_str -> out_str.append(vocab_str)
# lstm_char_index -> np.eyes[lstm_char_index] -> lstm_tmp
# reset(lstm_out), lstm_tmp.reshape(inp_dims) -> lstm_out
# vocab_str -> out_str.append(vocab_str)

model_inp_dims = (1,1,83)
printstr = ''

model_hardmax = lambda x: to_model_dims(np_onehot(np.argmax(x.flatten())))
num_to_input = lambda x: to_model_dims(train_x[x])
getchar = lambda x: str(vocab[np.asscalar(np.argmax(x,axis=0).flatten())])
out_to_string = lambda x: printstr + str(vocab[x])
vocab_to_int = lambda x: vocab_to_int[x]
np_onehot = lambda x: to_model_dims(np.eye(model_inp_dims[-1])[x])
to_model_dims = lambda x: x.astype('int8').reshape(1,1,83)

def print_loupe(louperun=1000):
    
    seed  = np.random.randint(0, train_x.shape[0])
    printstr = ''
    
    model_hardmax = lambda x: to_model_dims(np_onehot(np.argmax(x.flatten())))
    num_to_input = lambda x: to_model_dims(train_x[x])
    getchar = lambda x: str(vocab[np.asscalar(np.argmax(x,axis=0).flatten())])
    out_to_string = lambda x: printstr + str(vocab[x])
    vocab_to_int = lambda x: vocab_to_int[x]
    np_onehot = lambda x: to_model_dims(np.eye(model_inp_dims[-1])[x])
    to_model_dims = lambda x: x.astype('int8').reshape(1,1,83)
    
    lstm_in = num_to_input(seed)
    
    for _ in range(louperun):
        lstm_out = model_hardmax(lstm_pred_model.predict(lstm_in))
        printstr = out_to_string(np.argmax(lstm_out.flatten()))
        lstm_in = np_onehot(np.argmax(lstm_out.flatten()))
    
    print(printstr)
    

print_loupe()

### Reference Implementation Details - TF Model

In [ ]:
model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate).

In [ ]:
epochs = 10
# Print losses every N interations
print_every_n = 256

# Save every N iterations
save_every_n = 256

def safe_reload(obj, sess, checkpoint):
    if checkpoint is None:
        return None
    else:
        return obj(sess, checkpoint)

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    safe_reload(saver.restore,sess, tf.train.latest_checkpoint('checkpoints'))
    
    counter = 0
    epoch_losses = []
    for e in range(epochs):
        epochstart = time.time()
        epoch_losses = []
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
            epoch_losses.append(batch_loss)
    
        epochend = time.time()
        epochloss_avg = sum(epoch_losses)/len(epoch_losses)
        print('Epoch: {}/{}... '.format(e+1, epochs),
               '@ Training Step: {}... '.format(counter),
               'Epoch Averaged Training loss: {:.4f}... '.format(epochloss_avg),
               '{:.4f} sec/batch'.format((epochend-epochstart)))
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

In [ ]:
tf.train.get_checkpoint_state('checkpoints')

In [ ]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [ ]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [ ]:
tf.train.latest_checkpoint('checkpoints')

In [ ]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = 'checkpoints/i512_l512.ckpt'
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [ ]:
feed = train_x[0:128].reshape(128,1,83).astype('int8')
lstm_model.predict(feed, verbose=1)

Here, pass in the path to a checkpoint and sample from the network.

In [ ]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = LSTM_RNN.model
    with tf.Session() as sess:
        model_with_weights = model.load_weights(checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            preds = model_with_weights.preict(x, verbose=0)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [ ]:
checkpoint = 'checkpoints/keras_lstmm_weights-best.hdf5'
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)